# DWH

## Import

In [2]:
import os
from datetime import datetime
import pandas as pd
import requests
from bs4 import BeautifulSoup
from glob import glob

Eine importierte Stopword Liste aus Github wird verwendet:

In [3]:
stopwords_url = "https://raw.githubusercontent.com/solariz/german_stopwords/master/german_stopwords_full.txt"
stopwords_list = requests.get(stopwords_url, allow_redirects=True).text.split("\n")[9:]

In [4]:
# Pfade
DATA_LAKE_DIR = 'C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\ads_05\\ads_05_studienarbeit\\input\\data-lake'

## HTML und CSV Parsing Module

Mit den folgenden Funktionen lassen sich HTML-Dateien auslesen, deren Textinhalte bereinigen und für eine weitere Analyse in einem DataFrame zusammenführen. Dabei werden überflüssige HTML-Tags entfernt, Wörter gefiltert und gezählt sowie zusätzliche Informationen wie das Datum und der Zeitungsname hinzugefügt.

In [5]:
# Liest den Inhalt einer HTML-Datei
def read_html_file(file_name, encoding="utf-8"):
    with open(file_name, "r", encoding=encoding) as f:
        text = f.read()
    return text

# Extrahiert und bereinigt Text aus einer HTML-Datei
def process_html(text):
    text = BeautifulSoup(text, "html.parser").text  # Entfernt HTML-Tags
    items = text.replace("\n", " ").replace("\t", " ").lower().split(" ")  # Zerlegt in Wörter
    items = [i for i in items if len(i) > 2 and i not in stopwords_list]  # Filtert Stopwords
    return items

# Analysiert HTML-Dateien und erstellt ein DataFrame mit Wortzählungen
def parse_html(name, date, file_name, encoding):
    content = read_html_file(file_name, encoding)  # Lese Dateiinhalt
    soup = BeautifulSoup(content, "html.parser")  # Parsen des HTML-Inhalts
    text = soup.text  # Extrahiert Text
    items = process_html(text)  # Verarbeite Text
    item_count = pd.Series(items, dtype="object").value_counts()  # Zähle Wörter mit explizitem Datentyp
    count = item_count.to_frame()  # Konvertiere in DataFrame
    count.columns = ["count"]  # Spaltenname setzen
    count["word"] = count.index  # Wörter als Index
    count["date"] = date  # Datum hinzufügen
    count["paper"] = name  # Zeitung hinzufügen
    return count


Die naechste Funktion durchsucht ein angegebenes Verzeichnis nach CSV-Dateien, um darin hinterlegte Informationen (Dateiname, Codierung, Erscheinungsdatum und Zeitungsname) auszulesen. Anschließend werden die zugehörigen HTML-Dateien eingelesen, bereinigt und für eine weiterführende Analyse in einem DataFrame zusammengefasst.

In [6]:
def parse_csv_files_in_directory(directory):
    result_list = []  # Liste zum Speichern der Ergebnisse der HTML-Verarbeitung

    # Finde alle CSV-Dateien im Verzeichnis
    csv_filename_list = glob(os.path.join(directory, "*.csv"))  

    # Iteriere über jede CSV-Datei
    for csv_file in csv_filename_list:  
        df = pd.read_csv(csv_file)  # Lese die CSV-Datei in ein DataFrame
        df2 = df.dropna()  # Entferne Zeilen mit fehlenden Werten

        # Iteriere über jede Zeile im DataFrame
        for i, row in df2.iterrows():  
            name = row["name"]  # Extrahiere den Namen der Zeitung
            file_name = row["file_name"]  # Extrahiere den Dateinamen der HTML-Datei
            date = row["date"]  # Extrahiere das Datum
            encoding = row["encoding"]  # Extrahiere die Kodierung

            # Erstelle den absoluten Pfad zur HTML-Datei
            full_path = os.path.join(directory, file_name)  

            # Überprüfe, ob die HTML-Datei existiert
            if os.path.exists(full_path):  
                print(f"Datei gefunden: {full_path}")  # Gib eine Meldung aus, wenn die Datei gefunden wurde
                count = parse_html(name, date, full_path, encoding)  # Verarbeite die HTML-Datei
                result_list.append(count)  # Füge das Ergebnis zur Liste hinzu
            else:
                print(f"Datei nicht gefunden: {full_path}")  # Gib eine Meldung aus, wenn die Datei nicht gefunden wurde

    # Füge alle Ergebnisse zu einem DataFrame zusammen
    if result_list:  # Überprüfe, ob die Ergebnisliste nicht leer ist
        final_result_df = pd.concat(result_list, ignore_index=True)  # Kombiniere die DataFrames
        return final_result_df  # Gib den kombinierten DataFrame zurück
    else:
        return pd.DataFrame()  # Gib einen leeren DataFrame zurück, wenn keine Ergebnisse vorhanden sind

Funktion um ein Dataframe nach Schlüsselwörtern zu filtern:

In [7]:
# Filtert ein DataFrame basierend auf Schlüsselwörtern
def filter_words(df, column, keywords):
    def extract_keyword(text):
        for keyword in keywords:  # Prüfe jedes Schlüsselwort
            if keyword in text:
                return keyword  # Rückgabe des ersten Treffers
        return None  

    df[column] = df[column].apply(extract_keyword)  # Anwenden auf die Spalte
    return df


## HTML-Dateien Lesen und Verarbeiten

Dieser Abschnitt verarbeitet die aggregierten Wortzählungen aus den CSV-Dateien, filtert sie nach den definierten Parteien und bereitet das Ergebnis für weitere Analysen vor.

In [8]:
# Verarbeite Dateien und speichere das Ergebnis
result = parse_csv_files_in_directory('C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\ads_05\\ads_05_studienarbeit\\input\\data-lake')
print(result.head())  # Zeige die ersten Zeilen des DataFrames an

Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-sz.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-zeit.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-faz.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-heise.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-golem.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-tagesspiegel.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-taz.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit\input\data-lake\2021-04-01-abendblatt.html
Datei gefunden: C:\Users\hp\OneDrive\Desktop\DBU\ads_05\ads_05_studienarbeit

Eine Kopie erstellen:

In [9]:
copy = result.copy()  # Kopiere das DataFrame

Nach den Parteien filtern:

In [10]:
# Liste der zu filternden Parteien
keywords = ['spd', 'cdu', 'afd', 'fdp', 'grünen', 'linke', 'bsw']

# Wende die Filterung auf 'result' an
party_df = filter_words(copy, 'word', keywords)

# Zeige die ersten Zeilen des gefilterten DataFrames an
print(party_df.head())


   count  word        date paper
0     82  None  2021-04-01    sz
1     44  None  2021-04-01    sz
2     40  None  2021-04-01    sz
3     22  None  2021-04-01    sz
4     17  None  2021-04-01    sz


## Politbarometer-Daten einlesen

Dieser Codeabschnitt lädt die Wahlergebnisse aus einer CSV-Datei, benennt die Spalten um und formatiert das Datum.

In [12]:
# Vollständigen Pfad zur Excel-Datei angeben
file_path = "C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\ads_05\\ads_05_studienarbeit\\input\\politbarometer.xlsx"

# Lese die Excel-Datei ein und wähle das relevante Arbeitsblatt
wahlen_df = pd.read_excel(file_path, sheet_name='Tabelle1', usecols=[0, 1, 2, 3, 4, 5, 6, 7], parse_dates=[0])

# Benenne die Spalten für bessere Lesbarkeit um
wahlen_df.columns = ['date', 'CDU', 'SPD', 'Grüne', 'FDP', 'Linke', 'AfD', 'BSW']

# Konvertiere die 'date'-Spalte in das Datetime-Format
wahlen_df['date'] = pd.to_datetime(wahlen_df['date'], format='%Y-%m-%d', errors='coerce')

# Entferne Zeilen mit ungültigen Datumswerten (falls vorhanden)
wahlen_df = wahlen_df.dropna(subset=['date'])

# Zeige die ersten fünf Zeilen des DataFrames an
print(wahlen_df.head())


        date  CDU  SPD  Grüne   FDP  Linke  AfD  BSW
0 1991-02-15   43   35      7  10.0    2.0  NaN  NaN
1 1991-03-15   42   37      7  10.0    2.0  NaN  NaN
2 1991-04-26   41   38      7  10.0    2.0  NaN  NaN
3 1991-05-24   41   39      6  10.0    2.0  NaN  NaN
4 1991-06-14   40   40      6   9.0    2.0  NaN  NaN


## Als pickle speichern

In [16]:
TEMP_PATH = "C:\\Users\\hp\\OneDrive\\Desktop\\DBU\\ads_05\\ads_05_studienarbeit\\output\\pickles"
wahlen_df.to_pickle(os.path.join(TEMP_PATH, "politbarometer.pickle"))
party_df.to_pickle(os.path.join(TEMP_PATH, "parteien.pickle"))